In [2]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Input, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

2022-04-27 19:39:32.252833: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-27 19:39:32.252904: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /home/raghav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'ENTAILMENT': 0, 'CONTRADICTION': 1, 'NEUTRAL': 2}

In [4]:
# Function that reads data and parses data from file

fileName = 'SICK.txt'

with open(fileName, 'r') as f:
    Rows = [row.split('\t') for row in f.readlines()[1:]]

trainRows = [row for row in Rows if row[11]=='TRAIN\n']
testRows = [row for row in Rows if row[11]=='TEST\n']
validationRows = [row for row in Rows if row[11]=='TRIAL\n']

trainPremises = [extract(row[1]) for row in trainRows if row[3] in labels]
trainHypotheses = [extract(row[2]) for row in trainRows if row[3] in labels]
trainLabels = [labels[row[3]] for row in trainRows if row[3] in labels]

trainData = [trainPremises, trainHypotheses, trainLabels]

testPremises = [extract(row[1]) for row in testRows if row[3] in labels]
testHypotheses = [extract(row[2]) for row in testRows if row[3] in labels]
testLabels = [labels[row[3]] for row in testRows if row[3] in labels]

testData = [testPremises, testHypotheses, testLabels]

validationPremises = [extract(row[1]) for row in validationRows if row[3] in labels]
validationHypotheses = [extract(row[2]) for row in validationRows if row[3] in labels]
validationLabels = [labels[row[3]] for row in validationRows if row[3] in labels]

validationData = [validationPremises, validationHypotheses, validationLabels]

f.close()

In [5]:
# Hyperparameters

maxLen = 32
epochs = 1000
batchSize = 512
gloveDimension = 300
hiddenDimension = 100
regularization = 4e-6

In [6]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [7]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [8]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [9]:
# Convert the validation data to sequences as per the vocabulary
validationData[0] = tokenizer.texts_to_sequences(validationData[0])
validationData[1] = tokenizer.texts_to_sequences(validationData[1])

# Pad or trim all generated sequences to the same max sentence length
validationData[0] = pad_sequences(validationData[0], maxLen, padding='post')
validationData[1] = pad_sequences(validationData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
validationData[2] = tf.keras.utils.to_categorical(validationData[2], num_classes=3)

In [10]:
embeddingsDict = dict()
glove = open(r'glove.840B.300d.txt', encoding='utf8')

for line in glove:
    records = line.split()
    word = ''.join(records[:-300])
    vectorDimensions = asarray(records[-300:], dtype='float32')
    embeddingsDict[word] = vectorDimensions

glove.close()

In [11]:
# Iterate through the embeddings and store only those that are present in our vocabulary
embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec

In [12]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

convert = Dense(hiddenDimension, activation='tanh', input_shape=(gloveDimension,))

premInput = convert(premInput)
hypoInput = convert(hypoInput)

2022-04-27 19:43:12.677321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-27 19:43:12.687496: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-27 19:43:12.687885: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-04-27 19:43:12.687971: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-04-27 19:43:12.688040: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [13]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

#rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=hiddenDimension)
rnn = LSTM(hiddenDimension, dropout=0.2, recurrent_dropout=0.2)

In [14]:
# Apply batch normalization to the two input embeddings separately

premInput = rnn(premInput)
hypoInput = rnn(hypoInput)
premInput = tf.keras.layers.BatchNormalization()(premInput)
hypoInput = tf.keras.layers.BatchNormalization()(hypoInput)

In [15]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
joint = keras.layers.concatenate([premInput, hypoInput])
joint = Dropout(0.2)(joint)
for i in range(3):
    joint = Dense(2*hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(joint)
    joint = Dropout(0.2)(joint)
    joint = tf.keras.layers.BatchNormalization()(joint)

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function
pred = Dense(3, activation='softmax')(joint)

In [16]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=30)
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

Epoch 1/1000
9/9 [==============================] - 7s 742ms/step - loss: 0.4237 - accuracy: 0.8141 - val_loss: 0.7731 - val_accuracy: 0.6869
Epoch 2/1000
9/9 [==============================] - 7s 755ms/step - loss: 0.4027 - accuracy: 0.8281 - val_loss: 0.8560 - val_accuracy: 0.6909
Epoch 3/1000
9/9 [==============================] - 7s 770ms/step - loss: 0.4032 - accuracy: 0.8315 - val_loss: 0.8502 - val_accuracy: 0.6626
Epoch 4/1000
9/9 [==============================] - 7s 753ms/step - loss: 0.4121 - accuracy: 0.8254 - val_loss: 0.7565 - val_accuracy: 0.7172
Epoch 5/1000
9/9 [==============================] - 7s 752ms/step - loss: 0.3982 - accuracy: 0.8328 - val_loss: 0.8119 - val_accuracy: 0.6889
Epoch 6/1000
9/9 [==============================] - 7s 748ms/step - loss: 0.3998 - accuracy: 0.8355 - val_loss: 0.7744 - val_accuracy: 0.6929
Epoch 7/1000
9/9 [==============================] - 7s 771ms/step - loss: 0.3985 - accuracy: 0.8398 - val_loss: 0.7590 - val_accuracy: 0.6970
Epoch 

In [26]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

20/20 [==============================] - 2s 118ms/step - loss: 0.9689 - accuracy: 0.7161
Loss =  0.9689082503318787
Acc =  0.7160619497299194


In [29]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

Epoch 1/1000
9/9 [==============================] - 7s 733ms/step - loss: 0.1720 - accuracy: 0.9342 - val_loss: 1.1172 - val_accuracy: 0.7071
Epoch 2/1000
9/9 [==============================] - 7s 750ms/step - loss: 0.1756 - accuracy: 0.9385 - val_loss: 1.0985 - val_accuracy: 0.7273
Epoch 3/1000
9/9 [==============================] - 7s 753ms/step - loss: 0.1837 - accuracy: 0.9315 - val_loss: 1.0417 - val_accuracy: 0.7273
Epoch 4/1000
9/9 [==============================] - 8s 893ms/step - loss: 0.1630 - accuracy: 0.9390 - val_loss: 1.0075 - val_accuracy: 0.7354
Epoch 5/1000
9/9 [==============================] - 7s 737ms/step - loss: 0.1618 - accuracy: 0.9414 - val_loss: 0.9839 - val_accuracy: 0.7374
Epoch 6/1000
9/9 [==============================] - 7s 737ms/step - loss: 0.1733 - accuracy: 0.9365 - val_loss: 1.0187 - val_accuracy: 0.7293
Epoch 7/1000
9/9 [==============================] - 7s 729ms/step - loss: 0.1519 - accuracy: 0.9457 - val_loss: 1.0140 - val_accuracy: 0.7354
Epoch 

In [30]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

20/20 [==============================] - 2s 108ms/step - loss: 1.2887 - accuracy: 0.7265
Loss =  1.288654088973999
Acc =  0.726457417011261


In [31]:
model.save('./lstm50')

INFO:tensorflow:Assets written to: ./lstm50/assets


INFO:tensorflow:Assets written to: ./lstm50/assets


In [32]:
ccc = keras.models.load_model('./lstm50')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)

20/20 [==============================] - 3s 109ms/step - loss: 1.2887 - accuracy: 0.7265
0.726457417011261
